In [1]:
%matplotlib inline
import glob, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_rows = 200

plt.rcParams['figure.figsize'] = [20, 10]
plt.rc('font', size=20)



# Loading Dataset

## IOT Devices

The dataset has been extracted at `ROOT_PATH`, containing the data for each IOT device. There are 9 devices in total. There are folders for each IOT device containing `benign_traffic.csv` and further two folders for `gafgyt_attacks` and `mirai_attacks`

In [3]:
ROOT_PATH = r'/mnt/data/khiz/dataset/00442'

IOT_DEVS = [ 'Danmini_Doorbell',
             'Ecobee_Thermostat',
             #'Ennio_Doorbell',
             #'B120N10_Baby_Mon',
             '737E_Security_Cam',
             #'838_Security_Cam',
             #'Samsung_Webcam',
             #'1002_Security_Cam',
             #'1003_Security_Cam'
           ]

## Paths to CSVs
We create a python dictionary that contains all the paths to the CSVs for the respective IOT devices

In [4]:
PATHS_DICT = {}

for i in IOT_DEVS:
    PATHS_DICT[i] = {}
    print('[' + i + ']')
    iot_dir = os.path.join(ROOT_PATH, i)
    PATHS_DICT[i]['benign'] = os.path.join(iot_dir, 'benign_traffic.csv')
    print('  ', 'benign_traffic.csv')
    attacks = [ d for d in os.listdir(iot_dir)
                   if os.path.isdir( os.path.join(iot_dir, d)) ]
    for attack in attacks:
        attack_name = attack.split('_')[0]
        
        PATHS_DICT[i][attack_name] = {}
        attack_dir = os.path.join(iot_dir, attack)
        types = [ f for f in os.listdir(attack_dir)
                   if os.path.isfile( os.path.join(attack_dir, f)) ]
        print('  ', attack_name, ': ', types)
        for t in types:
            type_name = t.split('.')[0]
            PATHS_DICT[i][attack_name][type_name] = os.path.join(attack_dir,t)

[Danmini_Doorbell]
   benign_traffic.csv
   gafgyt :  ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']
   mirai :  ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']
[Ecobee_Thermostat]
   benign_traffic.csv
   gafgyt :  ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']
   mirai :  ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']
[737E_Security_Cam]
   benign_traffic.csv
   gafgyt :  ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']
   mirai :  ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']


## Loading CSVs into Pandas
We will create pandas dataframe for each botnet attack on each IOT device. For each datafram we will combine the attack traffic with the benign traffic add the following two columns:
- `traffic_type` : benign or attack (0 or 1)
- `attack_type`  : type of attack e.g, ack, scan etc.

In [15]:
nbaiot_dict = {}
for iot_dev in PATHS_DICT:
    nbaiot_dict[iot_dev] = {}
    b_df = pd.read_csv(PATHS_DICT[iot_dev]['benign'])
    b_df['traffic_type'] = 'benign'
    b_df['attack_type'] = 'benign'
    #nbaiot_dict[iot_dev] = nbaiot_dict[iot_dev].append(df)
    for botnet in [ b for b in PATHS_DICT[iot_dev] if b != 'benign']:
        nbaiot_dict[iot_dev][botnet] = pd.DataFrame()
        nbaiot_dict[iot_dev][botnet] = nbaiot_dict[iot_dev][botnet].append(b_df)
        for attack in PATHS_DICT[iot_dev][botnet]:
            a_df = pd.read_csv(PATHS_DICT[iot_dev][botnet][attack])
            a_df['traffic_type'] = 'attack'
            a_df['attack_type'] = attack
            nbaiot_dict[iot_dev][botnet] = nbaiot_dict[iot_dev][botnet].append(a_df)

In [16]:
for k in nbaiot_dict:
    print(k)
    for b in nbaiot_dict[k]:
        print('  ',b,':','<benign+attack>')

Danmini_Doorbell
   gafgyt : <benign+attack>
   mirai : <benign+attack>
Ecobee_Thermostat
   gafgyt : <benign+attack>
   mirai : <benign+attack>
737E_Security_Cam
   gafgyt : <benign+attack>
   mirai : <benign+attack>


In [14]:
#nbaiot_dict['Danmini_Doorbell']['mirai']

# Modeling

### Pre processing
Convert the target column `traffic_type` to integer 0 = benigh, 1 = attack 

In [18]:
for dev in nbaiot_dict:
    for botnet in nbaiot_dict[dev]:
        print(dev,botnet)
        nbaiot = nbaiot_dict[dev][botnet]

        nbaiot.loc[ nbaiot['traffic_type']=='attack','traffic_type' ]=1
        nbaiot.loc[ nbaiot['traffic_type']=='benign','traffic_type' ]=0
        nbaiot['traffic_type'] = nbaiot['traffic_type'].astype(int)
        nbaiot = nbaiot.drop(['attack_type'], axis=1)
        
        nbaiot_dict[dev][botnet] = nbaiot

Danmini_Doorbell gafgyt
Danmini_Doorbell mirai
Ecobee_Thermostat gafgyt
Ecobee_Thermostat mirai
737E_Security_Cam gafgyt
737E_Security_Cam mirai


### Classifiers and training functions

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


## Main Loop

In [ ]:
# classfiers = {
#     'KNN' : KNeighborsClassifier(),
#     'RFR' : RandomForestClassifier(),
#     'DTR' : DecisionTreeClassifier(),
#     'ADB' : AdaBoostClassifier(),
    
# }


# all_results = [
#     ['DEVICE','BOTNET', 'CLASSIFIER','TRN-SIZE', 'TST-SIZE-B',  'ACCURACY', 'PRECISION', 'RECALL', 'F1-SCORE', 'FALSE-P', 'FALSE-N' ]
# ]
# for dev in nbaiot_dict:
#     for botnet in nbaiot_dict[dev]:
#         nbaiot = nbaiot_dict[dev][botnet]
        
#         print("================", dev, "/", botnet, "================")

#         #for train_size_attack, train_size_benign in [ (10,10), (20,20), (40,40), (80,80), (200,200), (1000,1000) ]:
#         #for train_size_attack, train_size_benign in [ (100,100), (1000,1000), (20000,20000), (50000,50000), (100000,100000), (200000,200000) ]:
#         for test_size in [ 1-0.01, 1-0.05, 1-0.10, 1-0.20, 1-0.30, 1-0.33  ]:
#             print('train_size =',1-test_size)
#             print('test_size =',test_size)
#             #print('train_size_attack =',train_size_attack)
#             #print('train_size_benign =',train_size_benign)
        
# #             df_train_attack = nbaiot[ nbaiot['traffic_type'] == 1][:train_size_attack]
# #             print('train_size_attack =',len(df_train_attack.index))
# #             df_train_benign = nbaiot[ nbaiot['traffic_type'] == 0][:train_size_benign]
# #             print('train_size_benign =',len(df_train_benign.index))
# #             df_train = pd.concat([df_train_attack, df_train_benign])
            


# #             df_test_attack = nbaiot[ nbaiot['traffic_type'] == 1][train_size_attack+1:]
# #             print('test_size_attack =',len(df_test_attack.index))
# #             df_test_benign = nbaiot[ nbaiot['traffic_type'] == 0][train_size_benign+1:]
# #             print('test_size_benign =',len(df_test_benign.index))     
# #             df_test = pd.concat([df_test_attack, df_test_benign])
            
# #             X_train = df_train.drop(["attack_type", "traffic_type" ], axis=1)
# #             y_train = df_train["traffic_type"]
            
# #             X_test = df_test.drop(["attack_type", "traffic_type" ], axis=1)
# #             y_test = df_test["traffic_type"]

#             X = nbaiot.drop(["attack_type", "traffic_type" ], axis=1)
#             y = nbaiot['traffic_type']
#             X_train, X_test, y_train, y_test = train_test_split(
#                                         X, y, test_size=test_size, random_state=42)
            
#             for clf_name in classfiers:
                
#                 clf = classfiers[clf_name]
                
#                 clf.fit(X_train, y_train)
                
#                 y_preds = clf.predict(X_test)

#                 clrp = classification_report(y_test, y_preds, output_dict=True)
#                 conf_mat = confusion_matrix(y_test, y_preds)
#                 fp = conf_mat[0][1]
#                 fn = conf_mat[1][0]

#                 res = [
#                     dev,
#                     botnet,
#                     clf_name,
                    
#                     1-test_size,
#                     test_size,
                    
#                     clrp['accuracy'],
#                     clrp['macro avg']['precision'],
#                     clrp['macro avg']['recall'],
#                     clrp['macro avg']['f1-score'],
#                     fp,
#                     fn
#                 ]
#                 print(res)
#                 all_results.append(res)
#                 print("")


#         print("================================================================")
#         print("")
        

In [29]:
classfiers = {
    'KNN' : KNeighborsClassifier(),
    'RFR' : RandomForestClassifier(),
    'DTR' : DecisionTreeClassifier(),
    'ADB' : AdaBoostClassifier(),
    
}

all_results = [
    ['DEVICE','BOTNET', 'CLASSIFIER','TRAIN-SIZE-A', 'TEST-SIZE-B',  'ACCURACY', 'PRECISION', 'RECALL', 'F1-SCORE', 'FALSE-P', 'FALSE-N' ]
]



for dev in nbaiot_dict:
    for botnet in nbaiot_dict[dev]:
        nbaiot = nbaiot_dict[dev][botnet]
        
        print("================", dev, "/", botnet, "================")

        for train_attack_count in [ 
            10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000, 200000 ]:
            
            df_size = len(nbaiot.index)
            df_attack_count = len(nbaiot[ nbaiot['traffic_type'] == 1].index)
            df_benign_count = len(nbaiot[ nbaiot['traffic_type'] == 0].index)
            
            train_attack_pct = train_attack_count / df_attack_count
            train_benign_count = round(df_benign_count * train_attack_pct)
            
            df_attack = nbaiot[ nbaiot['traffic_type'] == 1]
            df_train_attack = df_attack.sample(train_attack_count)
            df_test_attack = df_attack.drop(df_train_attack.index)
            
            df_benign = nbaiot[ nbaiot['traffic_type'] == 0]
            df_train_benign = df_benign.sample(train_benign_count)
            df_test_benign = df_benign.drop(df_train_benign.index)
            
            df_train = pd.concat([ df_train_attack, df_train_benign ])
            df_test  = pd.concat([ df_test_attack,  df_test_benign  ]) 
            
            print('-----------------------------------------')            
            print('df_size =',df_size)
            print('df_attack_count =',df_attack_count)
            print('df_benign_count =',df_benign_count)
            print('train_attack_pct =',round(train_attack_pct,5))
            print('train_attack_count =',train_attack_count)
            print('train_benign_count =',train_benign_count)
            print('size of df_train =',len(df_train.index))
            print('size of df_test =',len(df_test.index))
            print('-----------------------------------------')
            print('')
            
            X_train = df_train.drop([ "traffic_type" ], axis=1)
            y_train = df_train["traffic_type"]
            
            X_test = df_test.drop([ "traffic_type" ], axis=1)
            y_test = df_test["traffic_type"]
            
            for clf_name in classfiers:
                
                clf = classfiers[clf_name]
                
                clf.fit(X_train, y_train)
                
                y_preds = clf.predict(X_test)

                clrp = classification_report(y_test, y_preds, output_dict=True)
                conf_mat = confusion_matrix(y_test, y_preds)
                fp = conf_mat[0][1]
                fn = conf_mat[1][0]

                res = [
                    dev,
                    botnet,
                    clf_name,
                    
                    train_attack_count,
                    train_benign_count,
                    
                    round(clrp['accuracy'],4),
                    round(clrp['macro avg']['precision'],4),
                    round(clrp['macro avg']['recall'],4),
                    round(clrp['macro avg']['f1-score'],4),
                    fp,
                    fn
                ]
                print(res)
                all_results.append(res)
                print("")


            


================ Danmini_Doorbell / gafgyt ================
-----------------------------------------
df_size = 366198
df_attack_count = 316650
df_benign_count = 49548
train_attack_pct = 3e-05
train_attack_count = 10
train_benign_count = 2
size of df_train = 12
size of df_test = 366168
-----------------------------------------



/home/khiz/.conda/envs/khiz-ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


['Danmini_Doorbell', 'gafgyt', 'KNN', 10, 2, 0.8647, 0.4323, 0.5, 0.4637, 49546, 0]

['Danmini_Doorbell', 'gafgyt', 'RFR', 10, 2, 0.967, 0.9712, 0.886, 0.9227, 11152, 922]

['Danmini_Doorbell', 'gafgyt', 'DTR', 10, 2, 0.9652, 0.9631, 0.8849, 0.9189, 11157, 1585]

['Danmini_Doorbell', 'gafgyt', 'ADB', 10, 2, 0.9672, 0.9714, 0.8867, 0.9232, 11082, 916]

-----------------------------------------
df_size = 366198
df_attack_count = 316650
df_benign_count = 49548
train_attack_pct = 0.00016
train_attack_count = 50
train_benign_count = 8
size of df_train = 58
size of df_test = 366002
-----------------------------------------

['Danmini_Doorbell', 'gafgyt', 'KNN', 50, 8, 0.9935, 0.9963, 0.976, 0.9859, 2373, 0]

['Danmini_Doorbell', 'gafgyt', 'RFR', 50, 8, 0.9967, 0.9905, 0.9956, 0.993, 298, 910]

['Danmini_Doorbell', 'gafgyt', 'DTR', 50, 8, 0.991, 0.9846, 0.9766, 0.9806, 2139, 1164]

['Danmini_Doorbell', 'gafgyt', 'ADB', 50, 8, 0.9905, 0.9845, 0.9748, 0.9796, 2314, 1148]

----------------------

/home/khiz/.conda/envs/khiz-ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


['Danmini_Doorbell', 'mirai', 'KNN', 10, 1, 0.9294, 0.4647, 0.5, 0.4817, 49547, 0]

['Danmini_Doorbell', 'mirai', 'RFR', 10, 1, 0.9458, 0.9724, 0.6163, 0.6745, 38024, 0]

['Danmini_Doorbell', 'mirai', 'DTR', 10, 1, 0.9844, 0.9095, 0.9916, 0.946, 0, 10954]

['Danmini_Doorbell', 'mirai', 'ADB', 10, 1, 0.9999, 0.9995, 1.0, 0.9997, 0, 51]

-----------------------------------------
df_size = 701648
df_attack_count = 652100
df_benign_count = 49548
train_attack_pct = 8e-05
train_attack_count = 50
train_benign_count = 4
size of df_train = 54
size of df_test = 701445
-----------------------------------------

['Danmini_Doorbell', 'mirai', 'KNN', 50, 4, 0.9646, 0.8376, 0.9548, 0.8854, 2809, 22023]

['Danmini_Doorbell', 'mirai', 'RFR', 50, 4, 0.9999, 0.9995, 1.0, 0.9997, 0, 51]

['Danmini_Doorbell', 'mirai', 'DTR', 50, 4, 0.9999, 0.9993, 0.9999, 0.9996, 0, 71]

['Danmini_Doorbell', 'mirai', 'ADB', 50, 4, 0.9999, 0.9993, 0.9999, 0.9996, 0, 71]

-----------------------------------------
df_size = 7

...

['Ecobee_Thermostat', 'mirai', 'KNN', 100, 3, 0.975, 0.4875, 0.5, 0.4937, 13110, 0]

['Ecobee_Thermostat', 'mirai', 'RFR', 100, 3, 0.9998, 0.9961, 0.9999, 0.998, 0, 102]

['Ecobee_Thermostat', 'mirai', 'DTR', 100, 3, 0.9971, 0.9482, 0.9985, 0.9719, 0, 1515]

['Ecobee_Thermostat', 'mirai', 'ADB', 100, 3, 0.9997, 0.9933, 0.9998, 0.9965, 0, 178]

-----------------------------------------
df_size = 525246
df_attack_count = 512133
df_benign_count = 13113
train_attack_pct = 0.00098
train_attack_count = 500
train_benign_count = 13
size of df_train = 513
size of df_test = 523227
-----------------------------------------

['Ecobee_Thermostat', 'mirai', 'KNN', 500, 13, 0.9988, 0.9973, 0.9779, 0.9874, 577, 53]

['Ecobee_Thermostat', 'mirai', 'RFR', 500, 13, 0.9999, 0.9972, 0.9999, 0.9985, 0, 75]

['Ecobee_Thermostat', 'mirai', 'DTR', 500, 13, 0.9995, 0.9911, 0.9998, 0.9954, 0, 237]

['Ecobee_Thermostat', 'mirai', 'ADB', 500, 13, 0.9988, 0.9974, 0.9779, 0.9874, 578, 51]

--------------------------

/home/khiz/.conda/envs/khiz-ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


['737E_Security_Cam', 'gafgyt', 'KNN', 10, 2, 0.8415, 0.4208, 0.5, 0.457, 62152, 0]

['737E_Security_Cam', 'gafgyt', 'RFR', 10, 2, 0.9447, 0.9592, 0.8317, 0.8803, 20744, 947]

['737E_Security_Cam', 'gafgyt', 'DTR', 10, 2, 0.8839, 0.7797, 0.8177, 0.7964, 17346, 28205]

['737E_Security_Cam', 'gafgyt', 'ADB', 10, 2, 0.953, 0.9536, 0.8659, 0.9027, 16252, 2195]

-----------------------------------------
df_size = 392250
df_attack_count = 330096
df_benign_count = 62154
train_attack_pct = 0.00015
train_attack_count = 50
train_benign_count = 9
size of df_train = 59
size of df_test = 392048
-----------------------------------------

['737E_Security_Cam', 'gafgyt', 'KNN', 50, 9, 0.9919, 0.9953, 0.9746, 0.9846, 3156, 0]

['737E_Security_Cam', 'gafgyt', 'RFR', 50, 9, 0.9666, 0.9719, 0.9015, 0.9324, 12045, 1067]

['737E_Security_Cam', 'gafgyt', 'DTR', 50, 9, 0.9678, 0.973, 0.9053, 0.9352, 11571, 1035]

['737E_Security_Cam', 'gafgyt', 'ADB', 50, 9, 0.9744, 0.9622, 0.9401, 0.9508, 6836, 3220]

------

/home/khiz/.conda/envs/khiz-ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


['737E_Security_Cam', 'mirai', 'KNN', 10, 1, 0.8752, 0.4376, 0.5, 0.4667, 62153, 0]

['737E_Security_Cam', 'mirai', 'RFR', 10, 1, 0.9326, 0.9643, 0.73, 0.7965, 33564, 0]

['737E_Security_Cam', 'mirai', 'DTR', 10, 1, 0.8737, 0.6969, 0.6221, 0.6458, 44331, 18569]

['737E_Security_Cam', 'mirai', 'ADB', 10, 1, 0.9769, 0.922, 0.9868, 0.951, 0, 11495]

-----------------------------------------
df_size = 498164
df_attack_count = 436010
df_benign_count = 62154
train_attack_pct = 0.00011
train_attack_count = 50
train_benign_count = 7
size of df_train = 57
size of df_test = 497950
-----------------------------------------

['737E_Security_Cam', 'mirai', 'KNN', 50, 7, 0.9924, 0.9957, 0.9697, 0.9822, 3765, 0]

['737E_Security_Cam', 'mirai', 'RFR', 50, 7, 0.9999, 0.9996, 0.9999, 0.9997, 2, 55]

['737E_Security_Cam', 'mirai', 'DTR', 50, 7, 0.9993, 0.9974, 0.9996, 0.9985, 0, 331]

['737E_Security_Cam', 'mirai', 'ADB', 50, 7, 0.9769, 0.9819, 0.9118, 0.9431, 10871, 612]

-------------------------------

In [30]:
res_df =pd.DataFrame(all_results, index=None)

res_df, res_df.columns = res_df[1:] , res_df.iloc[0]

res_df

,DEVICE,BOTNET,CLASSIFIER,TRAIN-SIZE-A,TEST-SIZE-B,ACCURACY,PRECISION,RECALL,F1-SCORE,FALSE-P,FALSE-N
1,Danmini_Doorbell,gafgyt,KNN,10,2,0.8647,0.4323,0.5,0.4637,49546,0
2,Danmini_Doorbell,gafgyt,RFR,10,2,0.967,0.9712,0.886,0.9227,11152,922
3,Danmini_Doorbell,gafgyt,DTR,10,2,0.9652,0.9631,0.8849,0.9189,11157,1585
4,Danmini_Doorbell,gafgyt,ADB,10,2,0.9672,0.9714,0.8867,0.9232,11082,916
5,Danmini_Doorbell,gafgyt,KNN,50,8,0.9935,0.9963,0.976,0.9859,2373,0
...,...,...,...,...,...,...,...,...,...,...,...
236,737E_Security_Cam,mirai,ADB,100000,14255,1,1,1,1,1,3
237,737E_Security_Cam,mirai,KNN,200000,28510,0.9907,0.9916,0.9875,0.9895,746,189
238,737E_Security_Cam,mirai,RFR,200000,28510,1,1,1,1,0,0
239,737E_Security_Cam,mirai,DTR,200000,28510,1,1,0.9999,1,4,0


In [31]:
# Write to csv
res_csv = res_df.to_csv(index=False)

print(res_csv, file=open('07_results.csv', 'w'))


### Load results from CSV

In [ ]:
res_df = pd.read_csv('02_results.csv')

res_df = res_df[['DEVICE', 'BOTNET', 'CLASSIFIER', 'ACCURACY', 'PRECISION', 'FALSE-P', 'FALSE-N' ]]

res_df

In [ ]:
res_df.set_index(['DEVICE', 'BOTNET', 'CLASSIFIER'], inplace=True)
res_df

In [ ]:
res_df0 = res_df.unstack(level=1)
res_df0

In [ ]:
res_df1 = res_df0.unstack(level=1)
res_df1

In [ ]:
res_df1.plot(kind='bar')